In [1]:
def decimalyear(stringdate):
    import datetime as dt
    from PyAstronomy import pyasl
    year=int(stringdate[0:4])
    month=int(stringdate[5:7])
    day=int(stringdate[8:10])
    hour=int(stringdate[11:13])
    minute=int(stringdate[14:])
    d=dt.datetime(year,month,day,hour,minute)
    dyear=pyasl.decimalYear(d)  
    return dyear

def datefromdec(dyear):
    from PyAstronomy import pyasl
    return pyasl.decimalYearGregorianDate(dyear,"yyyy-mm-dd hh:mm")

def fiveyears():
    import datetime as dt
    td=dt.date.today()
    tdtuple=td.timetuple()
    yr=tdtuple[0]+5
    fytuple=(yr,tdtuple[1],tdtuple[2])
    fy=dt.datetime(yr,tdtuple[1],tdtuple[2])
    return fy

In [2]:
import pandas as pd
import pickle
import numpy as np
from PyAstronomy import pyasl
import datetime as dt

craigslist_options_df=pd.read_pickle('craigslistdf.sav')
RealEstateModel=pickle.load(open('value_model.sav','rb'))
postdec=[]

for i in np.array(craigslist_options_df['posted']):
    postdec.append(decimalyear(i))

postdec
NormPredict=[]
for i in postdec:
     NormPredict.append(RealEstateModel.predict(np.array(i).reshape(-1,1)))

NormPredict=list(map(float, NormPredict))
NormPredict
fypredict=float(RealEstateModel.predict(np.array(pyasl.decimalYear(fiveyears())).reshape(-1,1)))
print(type(fypredict))#fypredict=map(float, fypredict)
d=NormPredict*craigslist_options_df['sqft']
craigslist_options_df["Current Price Prediction"]=d
craigslist_options_df['Percent Difference']=2*(np.absolute(craigslist_options_df["Current Price Prediction"]-craigslist_options_df['price'])/(craigslist_options_df['Current Price Prediction']+craigslist_options_df['price']))
craigslist_options_df['Five Years Date']=pyasl.decimalYear(fiveyears())
craigslist_options_df['Five Years Prediction']=fypredict*craigslist_options_df['sqft']*(1-craigslist_options_df['Percent Difference'])
top_picks=craigslist_options_df.sort_values(['Five Years Prediction'], ascending=False,na_position='last').head()
top_picks
#craigslist_options_df.columns
#RealEstateModel.predict(2020)

<class 'float'>


C:\Users\qtodz\Anaconda3\envs\REALESTATEPROJECT\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\qtodz\Anaconda3\envs\REALESTATEPROJECT\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


,posted,neighborhood,post title,number bedrooms,sqft,URL,price,Latitude,Longitude,Picture Link,Current Price Prediction,Percent Difference,Five Years Date,Five Years Prediction
4,2020-04-06 15:51,mission district,Well maintained office building for sale!,NaN,30000.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,13500000,37.764057,-122.418680,https://images.craigslist.org/00B0B_agTyaKYbYk...,1.696562e+07,0.227510,2025.290411,1.745736e+07
5,2020-03-30 10:08,noe valley,FOR SALE: 6 Unit Mixed-Use Property (4 Vacant ...,NaN,5848.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,4495000,37.747050,-122.426845,https://images.craigslist.org/00n0n_1n1JrGGlha...,3.302844e+06,0.305766,2025.290411,3.058286e+06
18,2020-04-02 19:11,pacific heights,Off Market - Fixer Upper Opportunity - Investo...,5,3900.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,2395000,37.785600,-122.435800,https://images.craigslist.org/00d0d_2xPKrkok4O...,2.203993e+06,0.083065,2025.290411,2.693816e+06
3,2020-04-06 15:50,mission district,Mixed use building on corner lot. Reduced price!,NaN,7500.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,2300000,37.763557,-122.418787,https://images.craigslist.org/00G0G_1cLkSBnbzA...,4.241404e+06,0.593574,2025.290411,2.296187e+06
8,2020-04-06 15:51,mission district,Beautiful commercial space on Mission!,NaN,2895.0,https://sfbay.craigslist.org/sfc/reb/d/san-fra...,1699000,37.750900,-122.415300,https://images.craigslist.org/00404_bsiREvKLCQ...,1.637182e+06,0.037059,2025.290411,2.099969e+06


In [3]:
print('Your Best Investment Is: ', top_picks.iloc[0,2])
print('The Current Price Is: $', top_picks.iloc[0,6])
print('Your Expected 5 year Earnings Are: $', top_picks.iloc[0,-1]-top_picks.iloc[0,6])
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from ipyleaflet import Map, Marker, basemaps, basemap_to_tiles

center = (top_picks.iloc[0,7], top_picks.iloc[0,8])
marker = Marker(location=center, draggable=False)
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom=15)


m.add_layer(marker);
display(m)
from IPython import display

display.Image(data=None,url=top_picks.iloc[0,9])

Your Best Investment Is:  Well maintained office building for sale!
The Current Price Is: $ 13500000
Your Expected 5 year Earnings Are: $ 3957362.4539217353


Map(center=['37.764057', '-122.418680'], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_t…